# OpenAIRE Data EDA

## Preamble

In [ ]:
%run notebook_preamble.ipy

pd.set_option('max_columns', 99)

In [ ]:
import seaborn as sns
import xmltodict
import pyjq
import boto3
import io

from src.visualization.visualize import pdf_cdf
from src.utils.misc_utils import print_nested_structure
from src.data.s3_transfer import get_files_from_s3
from src.data.openaire import parse_openaire_records

## Data Structure

### Projects

In [ ]:
BUCKET = 'im-eurito'
FOLDER = 'external/openaire/projectssoups'
KEY_PREFIX = 'soup'

In [ ]:
records = []
for file in get_files_from_s3(bucket=BUCKET, folder=FOLDER, key_prefix=KEY_PREFIX):
    records.extend(parse_openaire_records(file))

In [ ]:
df = pd.DataFrame().from_records(records)

In [ ]:
df.head()

### Publications

In [ ]:
BUCKET = 'im-eurito'
FOLDER = 'external/openaire/publicationssoups'
KEY_PREFIX = 'soup'

In [ ]:
def load_publications():
    for file in os.listdir(os.path.join(openaire_publication_data_path))[:5]:
        with open(os.path.join(openaire_publication_data_path, file), 'r') as f:
            data = f.read()
            for rec in parse_openaire_records(data, entity='publication'):
                yield rec

In [ ]:
# keeps timing out
# publication_records = []
# for file in get_files_from_s3(bucket=BUCKET, folder=FOLDER, key_prefix=KEY_PREFIX):
#     publication_records.extend(parse_openaire_records(file, entity='publication'))

In [ ]:
df_publications = pd.DataFrame().from_records(load_publications())

In [ ]:
df_publications.head()